# Session 9: Applied Data Analysis with Pandas

**Objective:** Build on your Pandas knowledge to perform more advanced analysis, including time series manipulation, data aggregation, and merging different datasets.

## Introduction

In Session 7, you learned the basics of Pandas DataFrames. Now, we'll explore the features that make Pandas an indispensable tool for quantitative finance. We will cover three key areas:

1.  **Time Series Analysis:** Financial data is almost always time-stamped. We'll learn how to leverage this to calculate things like moving averages.
2.  **Grouping and Aggregation:** How do you move from analyzing a single stock to analyzing an entire sector or industry? The `groupby` function is the answer.
3.  **Merging and Joining:** Financial analysis often requires combining data from multiple sources (e.g., price data from one file, company fundamentals from another). We'll learn how to join these datasets together.

In [1]:
import pandas as pd
from io import StringIO

## 1. Time Series Analysis

Pandas was built with financial time series in mind. To unlock its full power, you need to properly format your date columns.

In [2]:
# Sample time series data for a stock
csv_data = """Date,Close
2023-11-01,200.5
2023-11-02,201.2
2023-11-03,203.0
2023-11-06,202.5
2023-11-07,205.1
2023-11-08,204.8
2023-11-09,206.3
2023-11-10,208.0
"""

ts_df = pd.read_csv(StringIO(csv_data))
print("--- Original DataFrame Info ---")
ts_df.info()

# The 'Date' column is just an 'object' (a string). We need to convert it.
ts_df['Date'] = pd.to_datetime(ts_df['Date'])

# Now let's set the Date as the index of the DataFrame
ts_df.set_index('Date', inplace=True)

print("\n--- DataFrame Info After Conversion ---")
ts_df.info()
print("\n--- DataFrame with DateTimeIndex ---")
print(ts_df)

--- Original DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    8 non-null      object 
 1   Close   8 non-null      float64
dtypes: float64(1), object(1)
memory usage: 260.0+ bytes

--- DataFrame Info After Conversion ---
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8 entries, 2023-11-01 to 2023-11-10
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   8 non-null      float64
dtypes: float64(1)
memory usage: 128.0 bytes

--- DataFrame with DateTimeIndex ---
            Close
Date             
2023-11-01  200.5
2023-11-02  201.2
2023-11-03  203.0
2023-11-06  202.5
2023-11-07  205.1
2023-11-08  204.8
2023-11-09  206.3
2023-11-10  208.0


### Moving Averages
A moving average is used to smooth out price data to identify the direction of the trend. The `.rolling()` method is used for this.

In [3]:
# Calculate a 3-day simple moving average (SMA)
ts_df['SMA_3'] = ts_df['Close'].rolling(window=3).mean()

print(ts_df)
# Note: The first two values are NaN (Not a Number) because there aren't enough preceding data points to fill a 3-day window.

            Close       SMA_3
Date                         
2023-11-01  200.5         NaN
2023-11-02  201.2         NaN
2023-11-03  203.0  201.566667
2023-11-06  202.5  202.233333
2023-11-07  205.1  203.533333
2023-11-08  204.8  204.133333
2023-11-09  206.3  205.400000
2023-11-10  208.0  206.366667


## 2. Grouping and Aggregation

The `.groupby()` method lets you split data into groups based on some criteria, apply a function to each group independently, and combine the results.

In [4]:
# Sample data for multiple companies in different sectors
data = {
    'Ticker': ['AAPL', 'MSFT', 'JPM', 'BAC', 'GOOGL'],
    'Sector': ['Tech', 'Tech', 'Financials', 'Financials', 'Tech'],
    'Market_Cap_B': [2800, 2600, 450, 280, 1700],
    'Volume_M': [80, 50, 120, 150, 40]
}
sector_df = pd.DataFrame(data)

# Group the DataFrame by the 'Sector' column
sector_groups = sector_df.groupby('Sector')

# Now, apply aggregation functions to these groups
print("--- Average Market Cap by Sector ---")
print(sector_groups['Market_Cap_B'].mean())

print("\n--- Total Trading Volume by Sector ---")
print(sector_groups['Volume_M'].sum())

# You can do multiple aggregations at once with .agg()
print("\n--- Multiple Aggregations ---")
print(sector_groups.agg({
    'Market_Cap_B': ['mean', 'sum'],
    'Volume_M': 'mean'
}))

--- Average Market Cap by Sector ---
Sector
Financials     365.000000
Tech          2366.666667
Name: Market_Cap_B, dtype: float64

--- Total Trading Volume by Sector ---
Sector
Financials    270
Tech          170
Name: Volume_M, dtype: int64

--- Multiple Aggregations ---
           Market_Cap_B          Volume_M
                   mean   sum        mean
Sector                                   
Financials   365.000000   730  135.000000
Tech        2366.666667  7100   56.666667


## 3. Merging and Joining

`pd.merge()` allows you to combine DataFrames in a way that's similar to SQL joins.

In [5]:
# DataFrame 1: Price data
prices_df = pd.DataFrame({
    'Ticker': ['AAPL', 'MSFT', 'GOOGL'],
    'Price': [175, 310, 2800]
})

# DataFrame 2: Company fundamentals
fundamentals_df = pd.DataFrame({
    'Ticker': ['AAPL', 'MSFT', 'GOOGL'],
    'PE_Ratio': [30.1, 35.5, 28.9]
})

# Merge the two DataFrames on the 'Ticker' column
merged_df = pd.merge(prices_df, fundamentals_df, on='Ticker')

print(merged_df)

  Ticker  Price  PE_Ratio
0   AAPL    175      30.1
1   MSFT    310      35.5
2  GOOGL   2800      28.9


---

## Finance Exercise: Moving Average Signals and Sector Analysis

**Task:** You'll use a more extensive dataset to calculate moving averages for a trading signal and then perform a sector-wide analysis.

In [9]:
# Part 1: Moving Average Crossover Strategy
ma_csv_data = """Date,Close
2023-10-16,140.5
2023-10-17,141.2
2023-10-18,140.1
2023-10-19,142.0
2023-10-20,143.5
2023-10-23,142.8
2023-10-24,144.1
2023-10-25,145.9
2023-10-26,148.2
2023-10-27,150.0
2023-10-28,149.8
2023-10-29,148.7
2023-10-30,146.5
2023-10-31,144.0
2023-11-01,142.9
2023-11-02,141.4
2023-11-03,143.2
2023-11-06,142.5
2023-11-07,145.1
2023-11-08,144.8
2023-11-09,146.3
2023-11-10,148.0
"""
signal_df = pd.read_csv(StringIO(ma_csv_data), index_col='Date', parse_dates=True)

# Step 1: Calculate a short-term (3-day) and long-term (5-day) moving average
signal_df['SMA_3'] = signal_df['Close'].rolling(window=3).mean()
signal_df['SMA_5'] = signal_df['Close'].rolling(window=5).mean()

print("--- Data with Moving Averages ---")
print(signal_df)

# Step 2: Create a 'Signal' column.
# A 'Buy' signal is generated when the short-term SMA crosses ABOVE the long-term SMA.
# We can find this by checking two conditions:
# 1. Today's SMA_3 is greater than today's SMA_5
# 2. Yesterday's SMA_3 was less than yesterday's SMA_5

signal_df['Signal'] = 'Hold'
buy_condition = (signal_df['SMA_3'] > signal_df['SMA_5']) & (signal_df['SMA_3'].shift(1) < signal_df['SMA_5'].shift(1))
signal_df.loc[buy_condition, 'Signal'] = 'Buy'

print("\n--- Data with Trading Signal ---")
print(signal_df)

print("\n--- Days with a 'Buy' Signal ---")
print(signal_df[signal_df['Signal'] == 'Buy'])

--- Data with Moving Averages ---
            Close       SMA_3   SMA_5
Date                                 
2023-10-16  140.5         NaN     NaN
2023-10-17  141.2         NaN     NaN
2023-10-18  140.1  140.600000     NaN
2023-10-19  142.0  141.100000     NaN
2023-10-20  143.5  141.866667  141.46
2023-10-23  142.8  142.766667  141.92
2023-10-24  144.1  143.466667  142.50
2023-10-25  145.9  144.266667  143.66
2023-10-26  148.2  146.066667  144.90
2023-10-27  150.0  148.033333  146.20
2023-10-28  149.8  149.333333  147.60
2023-10-29  148.7  149.500000  148.52
2023-10-30  146.5  148.333333  148.64
2023-10-31  144.0  146.400000  147.80
2023-11-01  142.9  144.466667  146.38
2023-11-02  141.4  142.766667  144.70
2023-11-03  143.2  142.500000  143.60
2023-11-06  142.5  142.366667  142.80
2023-11-07  145.1  143.600000  143.02
2023-11-08  144.8  144.133333  143.40
2023-11-09  146.3  145.400000  144.38
2023-11-10  148.0  146.366667  145.34

--- Data with Trading Signal ---
            Close   